# Immersion Week Term 2: What is the Network Structure of the LIS MASc?

![poster](network_affects.jpeg)

![bridge](bridges.png)

Network theory emerged in the 18th century, when Leonhard Euler solved the famous "Seven Bridges of Königsberg" problem by inventing graph theory, reducing the city's layout to a set of nodes (land masses) and edges (bridges). For nearly 200 years, networks were studied primarily through this mathematical lens, focusing on regular, lattice-like structures. The field radically transformed in the mid-20th century with pioneers like Paul Erdős and Alfréd Rényi, who introduced random graph theory to model complex systems, and Stanley Milgram, whose "small-world" experiments revealed the social "six degrees of separation." However, the true revolution arrived in the late 1990s with digital computing. Researchers like Duncan Watts, Steven Strogatz, and Albert-László Barabási analyzed massive, real-world datasets—from the World Wide Web to metabolic pathways—and discovered that most natural networks are neither completely regular nor purely random, but instead follow universal laws, such as the scale-free structure where a few "hub" nodes hold the system together, fundamentally changing how we view the interconnected world.

In [ ]:
# --- Parameters ---
N = 12  # number of peripheral nodes (adjust as you like)

# --- 1. Build graphs ---
# Panopticon: star graph (one hub connected to all others, no other edges)
G_star = nx.star_graph(N)

# Fully connected: complete graph (every node connected to every other)
G_complete = nx.complete_graph(N + 1)  # same node count for fair comparison

# --- 2. Layout: circular with hub in centre ---
def circular_layout_with_center(n_peripheral):
    """Place node 0 at centre, rest on a circle."""
    pos = {0: (0.0, 0.0)}
    for i in range(1, n_peripheral + 1):
        angle = 2 * np.pi * (i - 1) / n_peripheral
        pos[i] = (np.cos(angle), np.sin(angle))
    return pos

pos = circular_layout_with_center(N)

# --- 3. Helper to build Plotly traces from a graph + positions ---
def make_edge_traces(G, pos, color="rgba(150,150,150,0.4)", width=1.2):
    edge_x, edge_y = [], []
    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
    return go.Scatter(
        x=edge_x, y=edge_y,
        mode="lines",
        line=dict(width=width, color=color),
        hoverinfo="none",
    )

def make_node_trace(G, pos, hub_color="#E63946", node_color="#457B9D", size=18):
    node_x = [pos[n][0] for n in G.nodes()]
    node_y = [pos[n][1] for n in G.nodes()]
    colors = [hub_color if n == 0 else node_color for n in G.nodes()]
    sizes  = [size * 1.6 if n == 0 else size for n in G.nodes()]
    return go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        marker=dict(size=sizes, color=colors,
                    line=dict(width=1.5, color="white")),
        hoverinfo="none",
    )

# --- 4. Assemble figure with subplots ---
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "<b>Panopticon (Star)</b><br><sup>Maximum hierarchy · single point of control</sup>",
        "<b>Fully Connected</b><br><sup>Zero hierarchy · total equality</sup>",
    ),
    horizontal_spacing=0.08,
)

# Panopticon (left)
fig.add_trace(make_edge_traces(G_star, pos, color="rgba(69,123,157,0.5)", width=2), row=1, col=1)
fig.add_trace(make_node_trace(G_star, pos), row=1, col=1)

# Fully connected (right) – use a different accent to visually distinguish
fig.add_trace(make_edge_traces(G_complete, pos, color="rgba(42,157,143,0.25)", width=1), row=1, col=2)
fig.add_trace(make_node_trace(G_complete, pos, hub_color="#2A9D8F", node_color="#2A9D8F", size=16), row=1, col=2)

# --- 5. Styling ---
axis_defaults = dict(
    showgrid=False, zeroline=False, showticklabels=False,
    showline=False, scaleratio=1,
)

fig.update_xaxes(**axis_defaults)
fig.update_yaxes(**axis_defaults, scaleanchor="x")   # col 1
fig.update_yaxes(**axis_defaults, scaleanchor="x2", row=1, col=2)  # col 2

fig.update_layout(
    width=1000, height=520,
    showlegend=False,
    margin=dict(l=30, r=30, t=80, b=30),
    paper_bgcolor="white",
    plot_bgcolor="white",
    title=dict(
        text="<b>Network Topology Comparison: Two Extremes</b>",
        x=0.5, font=dict(size=40),
    ),
    font=dict(family="Arial, sans-serif"),
)

# Add edge-count annotations
for col, G, label in [(1, G_star, "star"), (2, G_complete, "complete")]:
    n_edges = G.number_of_edges()
    fig.add_annotation(
        text=f"{G.number_of_nodes()} nodes · {n_edges} edges",
        xref=f"x{col if col > 1 else ''} domain", yref=f"y{col if col > 1 else ''} domain",
        x=0.5, y=-0.05, showarrow=False,
        font=dict(size=11, color="gray"),
    )

fig.show()

## Network Structure of the LIS MASc Cohort

This notebook builds **text-similarity networks** from the MASc cohort survey responses and uses them to introduce core concepts in network theory. Each student becomes a node; edges connect students whose writing is similar.

We construct **two networks**:
1. **Short-text network** — based on the ~120-word description of intellectual interests and values.
2. **Long-text network** — based on the longer reflective writing piece (e.g. Critical Reflection, capstone plan).

The notebook is designed to be **re-run as new survey responses arrive** — it reads whatever rows are present in the Excel file.

### Concepts covered
1. Nodes and edges (adjacency matrices)
2. Degree and degree distribution
3. Paths and distances
4. Clustering coefficient
5. Centrality and outliers

In [ ]:
# ── Setup & Data Loading ──────────────────────────────────────────────────────

!pip install networkx
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# ── Load data ─────────────────────────────────────────────────────────────────
DATA_FILE = "What's the network structure of the LIS MASc cohort_(1-28).xlsx"
df_raw = pd.read_excel(DATA_FILE)

# Column references (by position — resilient to minor name changes)
COL_PSEUDO = df_raw.columns[6]   # pseudonym
COL_SHORT  = df_raw.columns[7]   # short text (~120 words)
COL_LONG   = df_raw.columns[8]   # long reflective text
COL_QUESTION = df_raw.columns[9] # fun question

# Build a clean working dataframe
df = df_raw[[COL_PSEUDO, COL_SHORT, COL_LONG, COL_QUESTION]].copy()
df.columns = ['pseudonym', 'short_text', 'long_text', 'question']

# Fill missing text with empty strings
df['short_text'] = df['short_text'].fillna('').astype(str)
df['long_text']  = df['long_text'].fillna('').astype(str)

# Flag very short entries
df['short_len'] = df['short_text'].str.len()
df['long_len']  = df['long_text'].str.len()

print(f"Loaded {len(df)} respondents from {DATA_FILE}\n")
display(df[['pseudonym', 'short_len', 'long_len']].style
        .set_caption('Respondent summary')
        .bar(subset=['short_len', 'long_len'], color='#b3d1ff'))

## Text Preprocessing: TF-IDF and Cosine Similarity

### TF-IDF (Term Frequency – Inverse Document Frequency)

TF-IDF converts raw text into numerical vectors that capture how important each word is to a particular document *relative to the whole corpus*.

For a term $t$ in document $d$ within a corpus $D$:

$$\text{tf}(t, d) = \frac{\text{count of } t \text{ in } d}{\text{total terms in } d}$$

$$\text{idf}(t, D) = \log \frac{|D|}{1 + |\{d \in D : t \in d\}|}$$

$$\text{tf-idf}(t, d, D) = \text{tf}(t, d) \times \text{idf}(t, D)$$

Words that appear frequently in one document but rarely across the corpus receive high TF-IDF scores — they are *distinctive* to that document.



In [ ]:
# ── Toy Example: TF-IDF and Cosine Similarity ────────────────────────────────
# Three short documents about different topics.

toy_docs = [
    "The cat sat on the mat and the cat purred",
    "The dog chased the cat across the garden",
    "Python is a popular programming language for data science"
]
toy_labels = ["Doc A (cat)", "Doc B (cat & dog)", "Doc C (programming)"]

# Compute TF-IDF
toy_vec = TfidfVectorizer()
toy_tfidf = toy_vec.fit_transform(toy_docs)

# Show the TF-IDF matrix: each row is a document, each column is a word
tfidf_df = pd.DataFrame(
    toy_tfidf.toarray().round(3),
    index=toy_labels,
    columns=toy_vec.get_feature_names_out()
)
print("TF-IDF matrix (rows = documents, columns = words):")
display(tfidf_df.style.background_gradient(cmap='YlOrRd', axis=None)
        .format('{:.3f}'))

# Compute pairwise cosine similarity
toy_sim = cosine_similarity(toy_tfidf)
sim_df = pd.DataFrame(toy_sim.round(3), index=toy_labels, columns=toy_labels)

print("\nCosine similarity between documents:")
display(sim_df.style.background_gradient(cmap='Blues').format('{:.3f}'))

print("\nDocs A and B share the word 'cat' → moderate similarity (≈0.3).")
print("Doc C shares no words with A or B → similarity ≈ 0.")
print("Each document has similarity 1.0 with itself (the diagonal).")

In [ ]:
# ── Applied: TF-IDF & Cosine Similarity on Student Data ──────────────────────

def compute_similarity(texts, names):
    """Compute TF-IDF and pairwise cosine similarity for a list of texts."""
    vectorizer = TfidfVectorizer(stop_words='english', min_df=1)
    tfidf_matrix = vectorizer.fit_transform(texts)
    sim_matrix = cosine_similarity(tfidf_matrix)
    np.fill_diagonal(sim_matrix, 0)  # zero the diagonal — no self-loops in our network
    return pd.DataFrame(sim_matrix, index=names, columns=names)

names = df['pseudonym'].tolist()

sim_short = compute_similarity(df['short_text'], names)
sim_long  = compute_similarity(df['long_text'], names)

print("Short-text similarity matrix:")
print("(Diagonal is zeroed out — a student's similarity with themselves is always")
print(" 1.0, but self-loops are meaningless in our network, so we set it to 0.)\n")
display(sim_short.style.background_gradient(cmap='Blues', vmin=0, vmax=0.5)
        .format('{:.3f}').set_caption('Cosine similarity — short text'))

print("\nLong-text similarity matrix:")
display(sim_long.style.background_gradient(cmap='Oranges', vmin=0, vmax=0.5)
        .format('{:.3f}').set_caption('Cosine similarity — long text'))

## Network Construction

### Formal Definition

A **weighted undirected graph** is a triple $G = (V, E, w)$ where:
- $V$ is the set of **nodes** (here: students),
- $E \subseteq \binom{V}{2}$ is the set of **edges** (pairs of students),
- $w: E \to \mathbb{R}_{>0}$ assigns a positive **weight** to each edge (here: cosine similarity).

We apply a **threshold** $\tau$: an edge $(i, j)$ exists only if $\text{sim}(i, j) \geq \tau$. Adjusting $\tau$ controls how connected the network is — low thresholds create dense networks (many weak connections), high thresholds create sparse ones (only strong connections).

In [ ]:
# ── Toy Example: Threshold and Graph Construction ─────────────────────────────
# We'll use the 3-document similarity matrix from above to build tiny graphs
# at different thresholds.

toy_sim_matrix = cosine_similarity(toy_tfidf)
np.fill_diagonal(toy_sim_matrix, 0)
toy_sim_df = pd.DataFrame(toy_sim_matrix, index=toy_labels, columns=toy_labels)

print("Similarity matrix (diagonal zeroed):")
display(toy_sim_df.round(3))

for tau in [0.0, 0.15, 0.40]:
    G_toy = nx.Graph()
    G_toy.add_nodes_from(toy_labels)
    for i, n1 in enumerate(toy_labels):
        for j, n2 in enumerate(toy_labels):
            if j > i and toy_sim_df.iloc[i, j] >= tau:
                G_toy.add_edge(n1, n2, weight=round(toy_sim_df.iloc[i, j], 3))
    edges_str = ', '.join(f"{u}–{v} ({d['weight']:.2f})" for u, v, d in G_toy.edges(data=True))
    print(f"\n  τ = {tau:.2f} → {G_toy.number_of_edges()} edge(s): {edges_str or '(none)'}")

print("\nAt τ=0.0 Docs A & B are connected (they share 'cat').")
print("At τ=0.40 no edges survive — no pair is similar enough.")
print("The threshold acts as a dial between a dense and a sparse network.")

In [ ]:
# ── Helper functions (used throughout the notebook) ───────────────────────────

def build_graph(sim_df, threshold):
    """Build a weighted NetworkX graph from a similarity matrix at a given threshold."""
    G = nx.Graph()
    G.add_nodes_from(sim_df.index)
    for i, n1 in enumerate(sim_df.index):
        for j, n2 in enumerate(sim_df.columns):
            if j > i and sim_df.iloc[i, j] >= threshold:
                G.add_edge(n1, n2, weight=sim_df.iloc[i, j])
    return G


def plot_network(G, title, color='#4a90d9'):
    """Create a Plotly figure of the network using a spring layout."""
    pos = nx.spring_layout(G, seed=42, k=2/np.sqrt(max(len(G), 1)))

    # Edges
    edge_traces = []
    for u, v, d in G.edges(data=True):
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        weight = d.get('weight', 0)
        edge_traces.append(go.Scatter(
            x=[x0, x1, None], y=[y0, y1, None],
            mode='lines',
            line=dict(width=max(0.5, weight * 6), color='rgba(150,150,150,0.5)'),
            hoverinfo='text',
            text=f"{u} ↔ {v}: {weight:.3f}",
            showlegend=False
        ))

    # Nodes
    node_x = [pos[n][0] for n in G.nodes()]
    node_y = [pos[n][1] for n in G.nodes()]
    degrees = [G.degree(n) for n in G.nodes()]
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=list(G.nodes()),
        textposition='top center',
        textfont=dict(size=10),
        marker=dict(
            size=[max(15, 8 + d * 3) for d in degrees],
            color=degrees,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Degree'),
            line=dict(width=1, color='white')
        ),
        hoverinfo='text',
        hovertext=[f"{n}\nDegree: {G.degree(n)}" for n in G.nodes()],
        showlegend=False
    )

    fig = go.Figure(data=edge_traces + [node_trace])
    fig.update_layout(
        title=title,
        showlegend=False,
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=750, height=550,
        margin=dict(l=20, r=20, t=50, b=20),
        plot_bgcolor='white'
    )
    return fig


def build_and_plot(sim_df, threshold, label, color='#4a90d9'):
    """Build graph at threshold and plot it. Returns the graph."""
    G = build_graph(sim_df, threshold)
    fig = plot_network(G, f"{label} (threshold τ = {threshold:.2f})", color)
    fig.show()
    print(f"  Nodes: {G.number_of_nodes()}  |  Edges: {G.number_of_edges()}")
    return G

In [ ]:
# ── Applied: Student Networks ─────────────────────────────────────────────────
# Sensible defaults: short texts share more common vocabulary → higher threshold
THRESH_SHORT = 0.06
THRESH_LONG  = 0.06

print("=" * 60)
print("SHORT-TEXT NETWORK")
print("=" * 60)
G_short = build_and_plot(sim_short, THRESH_SHORT, 'Short-text network')

print("\n" + "=" * 60)
print("LONG-TEXT NETWORK")
print("=" * 60)
G_long = build_and_plot(sim_long, THRESH_LONG, 'Long-text network')

In [ ]:
# ── Threshold Explorer ────────────────────────────────────────────────────────
# Use this cell to experiment with different thresholds.
# Change the values below and re-run to see how the network changes.

EXPLORE_THRESHOLD = 0.1   # ← change this value
EXPLORE_WHICH = 'short'    # ← 'short' or 'long'

sim = sim_short if EXPLORE_WHICH == 'short' else sim_long
label = 'Short-text' if EXPLORE_WHICH == 'short' else 'Long-text'
G_explore = build_and_plot(sim, EXPLORE_THRESHOLD, f'{label} network (explorer)')

---

## 1. Nodes and Edges

### Formal Definitions

- A **node** (or vertex) $v \in V$ represents an entity — here, a student.
- An **edge** $e = \{u, v\} \in E$ represents a relationship — here, textual similarity above the threshold.
- The **adjacency matrix** $A$ of a graph with $n$ nodes is an $n \times n$ matrix where:

$$A_{ij} = \begin{cases} w(i,j) & \text{if } \{i,j\} \in E \\ 0 & \text{otherwise} \end{cases}$$

For unweighted graphs, $A_{ij} \in \{0, 1\}$. For weighted graphs (ours), $A_{ij}$ stores the cosine similarity.

In [ ]:
# ── Toy Example: Nodes, Edges, and Adjacency Matrix ──────────────────────────
# A small network of 5 people at a party. Edges represent who talked to whom.

G_party = nx.Graph()
G_party.add_nodes_from(['Alice', 'Bob', 'Carol', 'Dan', 'Eve'])
G_party.add_edges_from([
    ('Alice', 'Bob'),    # Alice and Bob chatted
    ('Alice', 'Carol'),  # Alice and Carol chatted
    ('Bob', 'Carol'),    # Bob and Carol chatted
    ('Carol', 'Dan'),    # Carol and Dan chatted
    # Eve talked to nobody — she's an isolated node
])

print("Party network:")
print(f"  Nodes (V): {list(G_party.nodes())}")
print(f"  Edges (E): {list(G_party.edges())}")
print(f"  |V| = {G_party.number_of_nodes()}, |E| = {G_party.number_of_edges()}")

# Adjacency matrix
party_nodes = ['Alice', 'Bob', 'Carol', 'Dan', 'Eve']
adj = nx.to_pandas_adjacency(G_party, nodelist=party_nodes, dtype=int)
print("\nAdjacency matrix (unweighted — 1 means connected, 0 means not):")
display(adj.style.background_gradient(cmap='Blues', vmin=0, vmax=1))

print("\nNotice: the matrix is symmetric (Alice→Bob = Bob→Alice) and the")
print("diagonal is 0 (no self-loops). Eve's row and column are all zeros — she's isolated.")

In [ ]:
# ── Applied: Nodes & Edges in Student Networks ───────────────────────────────

def show_adjacency_and_edges(G, label, cmap='Blues'):
    """Display adjacency matrix heatmap and edge list for a graph."""
    nodes = list(G.nodes())
    adj = nx.to_pandas_adjacency(G, nodelist=nodes)

    # Adjacency heatmap
    fig = px.imshow(
        adj.values, x=nodes, y=nodes,
        color_continuous_scale=cmap,
        title=f'{label} — Adjacency Matrix (weighted)',
        labels=dict(color='Similarity')
    )
    fig.update_layout(width=650, height=550)
    fig.show()

    # Edge list
    edges = [(u, v, f"{d['weight']:.3f}") for u, v, d in
             sorted(G.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)]
    if edges:
        print(f"\n{label} — Edge list ({len(edges)} edges), sorted by weight:")
        edge_df = pd.DataFrame(edges, columns=['Student A', 'Student B', 'Similarity'])
        display(edge_df)
    else:
        print(f"\n{label} — No edges at this threshold.")

    print(f"\n  Nodes: {G.number_of_nodes()}  |  Edges: {G.number_of_edges()}")


print("SHORT-TEXT NETWORK")
show_adjacency_and_edges(G_short, 'Short text', 'Blues')

print("\n" + "-" * 60)
print("\nLONG-TEXT NETWORK")
show_adjacency_and_edges(G_long, 'Long text', 'Oranges')

---

## 2. Degree and Degree Distribution

### Formal Definitions

- The **degree** of a node $v$ is the number of edges incident to it: $\deg(v) = |\{u : \{u,v\} \in E\}|$.
- The **weighted degree** (or **strength**) of a node is the sum of its edge weights: $s(v) = \sum_{u: \{u,v\} \in E} w(u,v)$.
- The **degree distribution** $P(k)$ gives the fraction of nodes with degree $k$.

In a random graph (Erdos–Renyi model $G(n, p)$), the degree distribution follows a binomial distribution centred at $(n-1)p$. Deviations from this tell us the network has non-random structure.

In [ ]:
# ── Toy Example: Degree ──────────────────────────────────────────────────────
# Using our party network from above.

print("Party network degrees:")
for person in party_nodes:
    nbrs = list(G_party.neighbors(person))
    print(f"  {person:6s}  deg = {G_party.degree(person)}  "
          f"(neighbours: {', '.join(nbrs) if nbrs else 'none'})")

print(f"\nMean degree: {np.mean([G_party.degree(n) for n in G_party.nodes()]):.1f}")
print("\nCarol has the highest degree (3) — she talked to the most people.")
print("Eve has degree 0 — she's isolated.")
print("\nIn a weighted network, 'strength' sums the edge weights instead of")
print("just counting edges — it distinguishes strong from weak connections.")

In [ ]:
# ── Applied: Degree & Degree Distribution in Student Networks ─────────────────

def degree_analysis(G, label, color):
    nodes = list(G.nodes())
    degrees = {n: G.degree(n) for n in nodes}
    strengths = {n: round(G.degree(n, weight='weight'), 3) for n in nodes}

    deg_df = pd.DataFrame({
        'Degree': degrees,
        'Strength (weighted degree)': strengths
    }).sort_values('Degree', ascending=False)

    # Bar chart of degree per student
    fig = px.bar(
        deg_df.reset_index(), x='index', y='Degree',
        title=f'{label} — Degree per student',
        labels={'index': 'Student', 'Degree': 'Degree'},
        color='Degree', color_continuous_scale='Viridis'
    )
    fig.update_layout(width=700, height=400)
    fig.show()

    # Degree distribution
    deg_values = list(degrees.values())
    fig2 = px.histogram(
        x=deg_values, nbins=max(deg_values) - min(deg_values) + 1 if deg_values else 1,
        title=f'{label} — Degree distribution',
        labels={'x': 'Degree', 'y': 'Count'},
        color_discrete_sequence=[color]
    )
    fig2.update_layout(width=600, height=350, bargap=0.1)
    fig2.show()

    # Erdos-Renyi comparison
    n = G.number_of_nodes()
    m = G.number_of_edges()
    max_edges = n * (n - 1) / 2
    p = m / max_edges if max_edges > 0 else 0
    expected_deg = (n - 1) * p
    print(f"  Edge density: {p:.3f}")
    print(f"  Expected degree in Erdos–Renyi G({n}, {p:.3f}): {expected_deg:.1f}")
    print(f"  Actual mean degree: {np.mean(deg_values):.1f}")
    print(f"  Actual std of degree: {np.std(deg_values):.2f}  "
          f"(Erdos–Renyi std ≈ {np.sqrt((n-1)*p*(1-p)):.2f})")

    display(deg_df)
    return deg_df


print("SHORT-TEXT NETWORK")
deg_short = degree_analysis(G_short, 'Short text', '#4a90d9')

print("\n" + "-" * 60)
print("\nLONG-TEXT NETWORK")
deg_long = degree_analysis(G_long, 'Long text', '#d97a4a')

---

## 3. Paths and Distances

### Formal Definitions

- A **path** from $u$ to $v$ is a sequence of edges connecting them: $u = v_0, v_1, \ldots, v_k = v$ where each $\{v_{i}, v_{i+1}\} \in E$.
- The **shortest path length** (or geodesic distance) $d(u, v)$ is the minimum number of edges in any path from $u$ to $v$.
- The **diameter** of a connected graph is the longest shortest path: $\text{diam}(G) = \max_{u,v \in V} d(u,v)$.
- The **average path length** is $\bar{L} = \frac{1}{n(n-1)} \sum_{u \neq v} d(u,v)$.
- A **connected component** is a maximal subgraph in which every pair of nodes is connected by a path. If $\tau$ is high enough, the network may fragment into separate components — students with no path between them.

In [ ]:
# ── Toy Example: Paths and Distances ─────────────────────────────────────────
# A slightly larger network: two triangles connected by a single bridge.
#
#   A ── B         E ── F
#    \ /           \ /
#     C ── bridge ── D

G_bridge = nx.Graph()
G_bridge.add_edges_from([
    ('A', 'B'), ('A', 'C'), ('B', 'C'),   # left triangle
    ('C', 'D'),                             # bridge
    ('D', 'E'), ('D', 'F'), ('E', 'F'),   # right triangle
])

# Shortest path from A to F
path_AF = nx.shortest_path(G_bridge, 'A', 'F')
print(f"Shortest path A → F: {' → '.join(path_AF)}  (length {len(path_AF)-1})")

path_AF2 = nx.shortest_path(G_bridge, 'A', 'E')
print(f"Shortest path A → E: {' → '.join(path_AF2)}  (length {len(path_AF2)-1})")

path_AB = nx.shortest_path(G_bridge, 'A', 'B')
print(f"Shortest path A → B: {' → '.join(path_AB)}  (length {len(path_AB)-1})")

print(f"\nDiameter (longest shortest path): {nx.diameter(G_bridge)}")
print(f"Average path length: {nx.average_shortest_path_length(G_bridge):.2f}")

# Connected components
print(f"Connected components: {nx.number_connected_components(G_bridge)}")

# Now remove the bridge and see what happens
G_broken = G_bridge.copy()
G_broken.remove_edge('C', 'D')
print(f"\nAfter removing the C–D bridge:")
print(f"  Connected components: {nx.number_connected_components(G_broken)}")
for i, comp in enumerate(nx.connected_components(G_broken)):
    print(f"  Component {i+1}: {sorted(comp)}")
print("  A and F are now unreachable from each other — no path exists.")

In [ ]:
# ── Applied: Paths & Distances in Student Networks ────────────────────────────

def path_analysis(G, label, cmap='Blues'):
    nodes = list(G.nodes())
    n = len(nodes)

    # Connected components
    components = list(nx.connected_components(G))
    print(f"  Connected components: {len(components)}")
    for i, comp in enumerate(sorted(components, key=len, reverse=True)):
        print(f"    Component {i+1} ({len(comp)} nodes): {sorted(comp)}")

    # Shortest path length matrix
    sp_matrix = np.full((n, n), np.inf)
    np.fill_diagonal(sp_matrix, 0)
    path_lengths = dict(nx.all_pairs_shortest_path_length(G))
    for i, n1 in enumerate(nodes):
        for j, n2 in enumerate(nodes):
            if n2 in path_lengths.get(n1, {}):
                sp_matrix[i, j] = path_lengths[n1][n2]

    # Replace inf with NaN for display
    sp_display = np.where(np.isinf(sp_matrix), np.nan, sp_matrix)

    fig = px.imshow(
        sp_display, x=nodes, y=nodes,
        color_continuous_scale=cmap,
        title=f'{label} — Shortest path lengths',
        labels=dict(color='Path length')
    )
    fig.update_layout(width=900, height=700)
    fig.show()

    # Diameter and average path length (per component)
    for i, comp in enumerate(sorted(components, key=len, reverse=True)):
        if len(comp) > 1:
            subG = G.subgraph(comp)
            diam = nx.diameter(subG)
            avg_pl = nx.average_shortest_path_length(subG)
            print(f"\n  Component {i+1}: diameter = {diam}, average path length = {avg_pl:.2f}")

    # Most/least reachable pairs
    finite_mask = np.isfinite(sp_matrix) & (sp_matrix > 0)
    if finite_mask.any():
        max_idx = np.unravel_index(np.where(finite_mask, sp_matrix, 0).argmax(), sp_matrix.shape)
        print(f"\n  Most distant reachable pair: {nodes[max_idx[0]]} ↔ {nodes[max_idx[1]]} "
              f"(path length {int(sp_matrix[max_idx])})")

    # Unreachable pairs
    unreachable = []
    for i in range(n):
        for j in range(i+1, n):
            if np.isinf(sp_matrix[i, j]):
                unreachable.append((nodes[i], nodes[j]))
    if unreachable:
        print(f"\n  Unreachable pairs ({len(unreachable)}):")
        for a, b in unreachable[:10]:
            print(f"    {a} ↔ {b}")
        if len(unreachable) > 10:
            print(f"    ... and {len(unreachable) - 10} more")


print("SHORT-TEXT NETWORK")
path_analysis(G_short, 'Short text', 'Blues')

print("\n" + "-" * 60)
print("\nLONG-TEXT NETWORK")
path_analysis(G_long, 'Long text', 'Oranges')

---

## 4. Clustering Coefficient

### Formal Definitions

- The **local clustering coefficient** of node $v$ measures how connected its neighbours are to each other:

$$C(v) = \frac{2 |\{\{u,w\} \in E : u,w \in N(v)\}|}{\deg(v)(\deg(v) - 1)}$$

where $N(v)$ is the set of neighbours of $v$. If $\deg(v) < 2$, we define $C(v) = 0$.

- The **global (average) clustering coefficient** is $\bar{C} = \frac{1}{n} \sum_{v \in V} C(v)$.
- **Transitivity** is the ratio of closed triplets to all triplets: $T = \frac{3 \times \text{triangles}}{\text{connected triples}}$.

High clustering means that "friends of friends tend to be friends" — or in our context, that students with similar writing tend to form tight clusters.

In [ ]:
# ── Toy Example: Clustering Coefficient ──────────────────────────────────────
# Using our bridge graph:  A─B─C (triangle) ── D─E─F (triangle)
#
# C has 3 neighbours: A, B, D.
#   Of the 3 possible edges among {A, B, D}, only A–B exists.
#   So C(C) = 2·1 / (3·2) = 1/3 ≈ 0.333
#
# A has 2 neighbours: B, C.
#   The only possible edge among {B, C} is B–C, and it exists.
#   So C(A) = 2·1 / (2·1) = 1.0

print("Bridge graph — local clustering coefficients:")
print()
for node in sorted(G_bridge.nodes()):
    nbrs = sorted(G_bridge.neighbors(node))
    cc = nx.clustering(G_bridge, node)
    print(f"  {node}: C = {cc:.3f}  (neighbours: {', '.join(nbrs)})")

print(f"\nWorked example for C:")
print(f"  Neighbours of C: A, B, D")
print(f"  Possible edges among them: A–B, A–D, B–D  (3 possible)")
print(f"  Actual edges among them: A–B only  (1 actual)")
print(f"  C(C) = 2×1 / (3×2) = 0.333")
print(f"\nWorked example for A:")
print(f"  Neighbours of A: B, C")
print(f"  Possible edges among them: B–C  (1 possible)")
print(f"  Actual edges among them: B–C  (1 actual)")
print(f"  C(A) = 2×1 / (2×1) = 1.000")
print(f"\nA sits inside a tight triangle → C = 1.")
print(f"C and D bridge two clusters → lower C (not all their friends know each other).")
print(f"\nAverage clustering: {nx.average_clustering(G_bridge):.3f}")
print(f"Transitivity: {nx.transitivity(G_bridge):.3f}")

In [ ]:
# ── Applied: Clustering Coefficient in Student Networks ───────────────────────

def clustering_analysis(G, label, color):
    cc = nx.clustering(G)
    cc_df = pd.DataFrame.from_dict(cc, orient='index', columns=['Clustering coeff.'])
    cc_df = cc_df.sort_values('Clustering coeff.', ascending=False)

    fig = px.bar(
        cc_df.reset_index(), x='index', y='Clustering coeff.',
        title=f'{label} — Local clustering coefficient per student',
        labels={'index': 'Student'},
        color='Clustering coeff.', color_continuous_scale='Viridis'
    )
    fig.update_layout(width=700, height=400)
    fig.show()

    avg_cc = nx.average_clustering(G)
    trans = nx.transitivity(G)
    print(f"  Average clustering coefficient: {avg_cc:.3f}")
    print(f"  Transitivity: {trans:.3f}")

    # Interpretation
    high_cc = cc_df[cc_df['Clustering coeff.'] > 0.5]
    if not high_cc.empty:
        print(f"\n  Students in tight clusters (C > 0.5): {', '.join(high_cc.index)}")

    bridges = cc_df[(cc_df['Clustering coeff.'] < 0.3) &
                    (cc_df['Clustering coeff.'] > 0)]
    if not bridges.empty:
        print(f"  Potential bridges (low C, but connected): {', '.join(bridges.index)}")

    display(cc_df)
    return cc_df


print("SHORT-TEXT NETWORK")
cc_short = clustering_analysis(G_short, 'Short text', '#4a90d9')

print("\n" + "-" * 60)
print("\nLONG-TEXT NETWORK")
cc_long = clustering_analysis(G_long, 'Long text', '#d97a4a')

---

## 5. Centrality and Outliers

### Formal Definitions

Centrality measures quantify how "important" or "influential" a node is within the network. Different measures capture different notions of importance:

- **Degree centrality**: $C_D(v) = \frac{\deg(v)}{n - 1}$ — fraction of possible connections realised. High degree centrality = connected to many others.

- **Betweenness centrality**: $C_B(v) = \sum_{s \neq v \neq t} \frac{\sigma_{st}(v)}{\sigma_{st}}$ where $\sigma_{st}$ is the number of shortest paths from $s$ to $t$, and $\sigma_{st}(v)$ is the number that pass through $v$. High betweenness = sits on many shortest paths (a "bridge" or "gatekeeper").

- **Closeness centrality**: $C_C(v) = \frac{n - 1}{\sum_{u \neq v} d(v, u)}$ — inverse of average distance to all other nodes. High closeness = can reach everyone quickly.

- **Eigenvector centrality**: solves $A\mathbf{x} = \lambda \mathbf{x}$ — a node is important if it is connected to other important nodes.

In [ ]:
# ── Toy Example: Centrality ──────────────────────────────────────────────────
# Using the bridge graph: A─B─C (triangle) ── D─E─F (triangle)
# This graph makes the differences between centrality measures vivid.

toy_cent = pd.DataFrame({
    'Degree':      nx.degree_centrality(G_bridge),
    'Betweenness': nx.betweenness_centrality(G_bridge),
    'Closeness':   nx.closeness_centrality(G_bridge),
    'Eigenvector':  nx.eigenvector_centrality(G_bridge),
}).round(3)

print("Bridge graph — centrality measures:")
display(toy_cent)

print("\nKey observations:")
print("  • Degree centrality: C and D have the most connections (3 each).")
print("  • Betweenness: C and D dominate — every shortest path between the two")
print("    triangles must pass through them. They are gatekeepers.")
print("  • Closeness: C and D are closest to everyone on average.")
print("  • Eigenvector: C and D score highest — they're connected to each other")
print("    (both important), while A is only connected to B and C.")
print("\nIn this symmetric graph, all four measures agree. In real networks,")
print("they often diverge — a node can be a bridge (high betweenness) without")
print("having many connections (low degree).")

In [ ]:
# ── Applied: Centrality & Outliers in Student Networks ────────────────────────

def centrality_analysis(G, sim_df, label, color):
    nodes = list(G.nodes())

    # Compute centralities
    deg_c = nx.degree_centrality(G)
    bet_c = nx.betweenness_centrality(G)
    # Closeness: handle disconnected graphs
    clo_c = {}
    for comp in nx.connected_components(G):
        subG = G.subgraph(comp)
        if len(comp) > 1:
            sub_clo = nx.closeness_centrality(subG)
            clo_c.update(sub_clo)
        else:
            for n in comp:
                clo_c[n] = 0.0
    # Eigenvector centrality: handle disconnected graphs
    eig_c = {}
    for comp in nx.connected_components(G):
        subG = G.subgraph(comp)
        if len(comp) > 1:
            try:
                sub_eig = nx.eigenvector_centrality(subG, max_iter=1000)
            except nx.PowerIterationFailedConvergence:
                sub_eig = {n: 0.0 for n in comp}
            eig_c.update(sub_eig)
        else:
            for n in comp:
                eig_c[n] = 0.0

    cent_df = pd.DataFrame({
        'Degree': deg_c,
        'Betweenness': bet_c,
        'Closeness': clo_c,
        'Eigenvector': eig_c
    }).round(3)

    # Bar chart comparison
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=['Degree centrality', 'Betweenness centrality',
                                        'Closeness centrality', 'Eigenvector centrality'])
    for idx, col in enumerate(cent_df.columns):
        row, c = divmod(idx, 2)
        sorted_df = cent_df.sort_values(col, ascending=False)
        fig.add_trace(
            go.Bar(x=sorted_df.index, y=sorted_df[col], name=col,
                   marker_color=color, showlegend=False),
            row=row+1, col=c+1
        )
    fig.update_layout(title=f'{label} — Centrality measures', width=800, height=600)
    fig.show()

    # Network with node size ∝ eigenvector centrality
    pos = nx.spring_layout(G, seed=42, k=2/np.sqrt(max(len(G), 1)))
    edge_traces = []
    for u, v, d in G.edges(data=True):
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_traces.append(go.Scatter(
            x=[x0, x1, None], y=[y0, y1, None],
            mode='lines',
            line=dict(width=1, color='rgba(150,150,150,0.4)'),
            hoverinfo='none', showlegend=False
        ))

    eig_vals = [eig_c.get(n, 0) for n in G.nodes()]
    max_eig = max(eig_vals) if max(eig_vals) > 0 else 1
    node_sizes = [max(12, 40 * e / max_eig) for e in eig_vals]

    node_trace = go.Scatter(
        x=[pos[n][0] for n in G.nodes()],
        y=[pos[n][1] for n in G.nodes()],
        mode='markers+text',
        text=list(G.nodes()),
        textposition='top center', textfont=dict(size=9),
        marker=dict(
            size=node_sizes,
            color=eig_vals,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Eigenvector<br>centrality'),
            line=dict(width=1, color='white')
        ),
        hovertext=[f"{n}\nEigenvector: {eig_c.get(n,0):.3f}\n"
                   f"Degree: {deg_c[n]:.3f}\nBetweenness: {bet_c[n]:.3f}"
                   for n in G.nodes()],
        hoverinfo='text', showlegend=False
    )

    fig2 = go.Figure(data=edge_traces + [node_trace])
    fig2.update_layout(
        title=f'{label} — Node size ∝ eigenvector centrality',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=750, height=550, margin=dict(l=20, r=20, t=50, b=20),
        plot_bgcolor='white'
    )
    fig2.show()

    # Identify hubs and outliers
    print(f"\n  Most central (by eigenvector): "
          f"{cent_df['Eigenvector'].idxmax()} ({cent_df['Eigenvector'].max():.3f})")
    print(f"  Most central (by betweenness): "
          f"{cent_df['Betweenness'].idxmax()} ({cent_df['Betweenness'].max():.3f})")

    isolated = [n for n in G.nodes() if G.degree(n) == 0]
    if isolated:
        print(f"  Isolated nodes (degree 0): {', '.join(isolated)}")

    peripheral = cent_df[cent_df['Degree'] <= cent_df['Degree'].quantile(0.25)]
    if not peripheral.empty:
        print(f"  Peripheral nodes (bottom 25% degree): {', '.join(peripheral.index)}")

    display(cent_df.sort_values('Eigenvector', ascending=False))
    return cent_df


print("SHORT-TEXT NETWORK")
cent_short = centrality_analysis(G_short, sim_short, 'Short text', '#4a90d9')

print("\n" + "-" * 60)
print("\nLONG-TEXT NETWORK")
cent_long = centrality_analysis(G_long, sim_long, 'Long text', '#d97a4a')

---

## Comparison & Reflection

In [ ]:
# ── Comparison & Reflection ──────────────────────────────────────────────────

# ── Side-by-side summary ─────────────────────────────────────────────────────
comparison = pd.DataFrame({
    'Short-text network': {
        'Nodes': G_short.number_of_nodes(),
        'Edges': G_short.number_of_edges(),
        'Threshold': THRESH_SHORT,
        'Density': f"{nx.density(G_short):.3f}",
        'Components': nx.number_connected_components(G_short),
        'Avg clustering': f"{nx.average_clustering(G_short):.3f}",
        'Transitivity': f"{nx.transitivity(G_short):.3f}",
    },
    'Long-text network': {
        'Nodes': G_long.number_of_nodes(),
        'Edges': G_long.number_of_edges(),
        'Threshold': THRESH_LONG,
        'Density': f"{nx.density(G_long):.3f}",
        'Components': nx.number_connected_components(G_long),
        'Avg clustering': f"{nx.average_clustering(G_long):.3f}",
        'Transitivity': f"{nx.transitivity(G_long):.3f}",
    }
})
print("Network Comparison")
display(comparison)

# ── Do the same students occupy similar positions? ────────────────────────────
print("\n── Centrality correlation across networks ──")
common_nodes = sorted(set(cent_short.index) & set(cent_long.index))
for measure in ['Degree', 'Betweenness', 'Closeness', 'Eigenvector']:
    s = cent_short.loc[common_nodes, measure]
    l = cent_long.loc[common_nodes, measure]
    corr = s.corr(l)
    print(f"  {measure:15s} Pearson r = {corr:+.3f}")

print("\nPositive correlations suggest students occupy similar positions in both ")
print("networks — their short self-description aligns with their longer writing.")
print("Low or negative correlations suggest the two texts reveal different facets ")
print("of intellectual identity.")

# ── Rank comparison ──────────────────────────────────────────────────────────
rank_df = pd.DataFrame({
    'Short-text eigenvector rank': cent_short.loc[common_nodes, 'Eigenvector']
        .rank(ascending=False).astype(int),
    'Long-text eigenvector rank': cent_long.loc[common_nodes, 'Eigenvector']
        .rank(ascending=False).astype(int),
})
rank_df['Rank change'] = rank_df.iloc[:, 0] - rank_df.iloc[:, 1]
display(rank_df.sort_values('Short-text eigenvector rank'))

# ── Fun appendix: student-submitted questions ────────────────────────────────
print("\n" + "=" * 60)
print("APPENDIX: Questions submitted by cohort members")
print("=" * 60)
for _, row in df.iterrows():
    q = row['question']
    if pd.notna(q) and str(q).strip() and str(q).strip().lower() not in ('nan', 'answer'):
        print(f"  {row['pseudonym']}: \"{q}\"")